# Panre with Law

In [51]:
LAW_TEXT_DIR = "../../data/law_text"

In [1]:
import json
import glob
import natsort
import difflib
import pandas as pd

In [52]:
law_text_list = []

for i in natsort.natsorted(glob.glob(f'{LAW_TEXT_DIR}/*.json')):
    with open(i, 'r') as f:
        law_text = json.load(f)
        law_text_list.append(law_text)

In [3]:
law_list = pd.read_csv('../../data/panre/law.csv')

In [4]:
def find_law_file_by_name(df, law_name):

    """
    법령명을 기준으로 법령이 저장된 file명을 찾는 함수
    input:
    - df: 법령 데이터프레임
    - law_name: 찾고자 하는 법령명

    output:
    - 법령이 저장된 파일명
    """
    
    if law_name == "헌법":
        law_name = "대한민국헌법"

    # 법령명을 기준으로 데이터프레임에서 해당하는 행을 검색
    result = df[df['법령명한글'] == law_name]['법령ID']

    if result.empty:
        matches = difflib.get_close_matches(law_name, df['법령명한글'], n=1, cutoff=0.3)
        # print(f"'{law_name}'과 매칭되는 정확한 법령명이 없어 유사한 법령명 '{matches[0]}'으로 대체합니다.")
        law_name = matches[0]
    
    result = df[df['법령명한글'] == law_name]['법령ID']
    
    # 결과값이 존재할 경우
    if not result.empty:
        # 법령ID를 숫자 6자리로 변환하여 반환
        return format(result.iloc[0], '06') + ".json"  # 첫 번째 매칭 결과 반환
    else:
        # 일치하는 법령명이 없는 경우 예외 처리
        raise ValueError("해당 법령명이 데이터베이스에 없습니다.")

In [5]:
def get_law_text_by_law_name(directory, law_name):

    """
    법령명을 기준으로 법령 내용을 반환하는 함수
    input:
    - directory: 법령 파일(json) 저장 경로
    - law_name: 법령명

    output:
    - 법령 전문
    """
    
    # 법령명을 기준으로 법령 파일을 찾음
    law_file = find_law_file_by_name(law_list, law_name)
    
    # 법령 파일을 읽어서 법령 내용을 반환
    with open(f"{directory}/{law_file}", 'r') as f:
        law_text = json.load(f)
        return law_text

In [6]:
def number_to_circle(num):

    """
    숫자를 원문자로 변환하는 함수 (항번호는 원문자로 이루어져 있음)
    input:
    - num: 변환하고자 하는 숫자 (1~50, None)

    output:
    - 원문자로 변환된 문자열
    """
    
    if type(num) != int:
        return None
    # 원문자 시작 코드 포인트는 9312 (①)
    base_code = 9311  # '①' 이전 문자의 유니코드 값
    if 1 <= num <= 20:
        return chr(base_code + num)
    elif 21 <= num <= 35:
        return chr(base_code + num + 3549)
    elif 36 <= num <= 50:
        return chr(base_code + num + 3549 + 81)

# 예시 사용
for i in range(50):
    print(number_to_circle(i+1), end="")

①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮⑯⑰⑱⑲⑳㉑㉒㉓㉔㉕㉖㉗㉘㉙㉚㉛㉜㉝㉞㉟㊱㊲㊳㊴㊵㊶㊷㊸㊹㊺㊻㊼㊽㊾㊿

In [28]:
def get_jomun_by_law_text(law_text, jomun_num):

    """
    법령 전문에서 조문 번호에 해당하는 조문 내용을 반환하는 함수
    input:
    - law_text: 법령 전문
    - jomun_num: 찾고자 하는 조문 번호

    output:
    - 조문 내용
    """
    
    for jomun in law_text['조문']:
        if jomun['조문번호'] == str(jomun_num) and jomun['조문여부'] == '조문':
            return jomun['조문내용']
    raise ValueError(f"해당 조문 ({jomun_num}조)이 없습니다.")

In [29]:
def get_hang_by_law_text(law_text, jomun_num, hang_num):
    
    """
    법령 전문에서 조문 번호와 항 번호에 해당하는 항 내용을 반환하는 함수
    input:
    - law_text: 법령 전문
    - jomun_num: 찾고자 하는 조문 번호
    - hang_num: 찾고자 하는 항 번호

    output:
    - 항 내용
    """
    
    hang_num = number_to_circle(hang_num)
    for jomun in law_text['조문']:
        if jomun['조문번호'] == str(jomun_num) and jomun['조문여부'] == '조문':
            if '항' in jomun:
                for hang in jomun['항']:
                    if hang['항번호'] == hang_num:
                        return hang['항내용']
            else:
                raise ValueError(f"해당 조문 ({jomun_num}조)에는 항이 없습니다.")

In [30]:
def get_ho_by_law_text(law_text, jomun_num, hang_num, ho_num):
    
    """
    법령 전문에서 조문 번호, 항 번호, 호 번호에 해당하는 호 내용을 반환하는 함수
    input:
    - law_text: 법령 전문
    - jomun_num: 찾고자 하는 조문 번호
    - hang_num: 찾고자 하는 항 번호
    - ho_num: 찾고자 하는 호 번호

    output:
    - 호 내용
    """
        
    hang_num = number_to_circle(hang_num)
    ho_num = str(ho_num) + "."
    for jomun in law_text['조문']:
        if jomun['조문번호'] == str(jomun_num) and jomun['조문여부'] == '조문':
            if '항' in jomun:
                for hang in jomun['항']:
                    if hang['항번호'] == hang_num:
                        for ho in hang['호']:
                            if ho['호번호'] == ho_num:
                                return ho['호내용']
            else:
                raise ValueError(f"해당 조문 항({jomun_num}조 {hang_num})에는 호가 없습니다.")

In [31]:
def get_ho_by_hang(hang):
    
    """
    항에서 호 내용을 반환하는 함수
    input:
    - hang: 항

    output:
    - 호 내용
    """

    hos = []
    if '호' in hang:
        for ho in hang['호']:
            hos.append(ho['호내용'])
    return hos


def get_hang_by_jomun(jomun):

    """
    조문에서 항 내용을 반환하는 함수
    input:
    - jomun: 조문

    output:
    - 항 내용
    """

    hangs = []
    if '항' in jomun:
        for hang in jomun['항']:
            hos = get_ho_by_hang(hang)
            hangs.append((hang['항내용'], hos))
    return hangs

In [32]:
def get_contents_by_law_text(law_text, jomun_num=None, hang_num=None):

    """
    법령 전문에서 조문, 항, 호에 해당하는 내용을 반환하는 함수
    input:
    - law_text: 법령 전문
    - jomun_num: 찾고자 하는 조문 번호
    - hang_num: 찾고자 하는 항 번호

    output:
    - 조문 내용 또는 항 내용 또는 호 내용
    """

    
    # 조문 번호를 원하는 경우
    if jomun_num is not None:
        jomun_num = str(jomun_num)
        for jomun in law_text['조문']:
            if jomun['조문번호'] == jomun_num and jomun['조문여부'] == '조문':
                if hang_num is None:
                    # 모든 항과 호를 가져온다.
                    hangs = get_hang_by_jomun(jomun)
                    return jomun['조문내용'], hangs
                else:
                    # 특정 항의 호만 가져온다.
                    hang_num = number_to_circle(hang_num)
                    for hang in jomun.get('항', []):
                        if hang['항번호'] == hang_num:
                            hos = get_ho_by_hang(hang)
                            return hang['항내용'], hos
        raise ValueError(f"{jomun_num}조의 정보가 없거나, 해당 조에 {hang_num}항이 없습니다.")
    else:
        raise ValueError("조문 번호는 필수입니다.")

In [33]:
def find_law_section_by_content(law_text_list, search_content):
    """
    법령 전문에서 주어진 내용에 해당하는 조문 또는 항을 찾는 함수
    input:
    - law_text_list: 법령 전문 리스트
    - search_content: 찾고자 하는 내용

    output:
    - 발견된 조문 또는 항의 정보
    """
    
    law_name, jomun_num, hang_num, ho_num = None, None, None, None

    for law_text in law_text_list:
        # 조문 검사
        for jomun in law_text['조문']:
            jomun_content = jomun.get('조문내용', '')
            if search_content in jomun_content:
                jomun_num = jomun['조문번호']
                law_id = law_text['법령ID']
                law_name = law_text['법령명_한글']

        # 항 검사
        for jomun in law_text['조문']:
            if '항' in jomun:
                for hang in jomun.get('항', []):
                    hang_content = hang.get('항내용', '')
                    if hang_content is None:
                        pass
                    elif search_content in hang_content:
                        hang_num = hang['항번호']
                        jomun_num = jomun['조문번호']
                        law_id = law_text['법령ID']
                        law_name = law_text['법령명_한글']
                    
        # 호 검사
        for jomun in law_text['조문']:
            if '항' in jomun:
                for hang in jomun.get('항', []):
                    if '호' in hang:
                        for ho in hang.get('호', []):
                            ho_content = ho.get('호내용', '')
                            if ho_content is None:
                                pass
                            elif search_content in ho_content:
                                ho_num = ho['호번호']
                                hang_num = hang['항번호']
                                jomun_num = jomun['조문번호']
                                law_id = law_text['법령ID']
                                law_name = law_text['법령명_한글']
    
    return law_id, law_name, jomun_num, hang_num, ho_num

## Test Code

In [290]:
find_law_file_by_name(law_list, '구 자본시장과 금융투자업에 관한 법률')

'구 자본시장과 금융투자업에 관한 법률'과 매칭되는 정확한 법령명이 없어 유사한 법령명 '자본시장과 금융투자업에 관한 법률'으로 대체합니다.


'010513.json'

In [195]:
get_law_text_by_law_name('../../data/panre/law_text_cleansed', '헌법')

{'법령ID': '001444',
 '공포일자': '19871029',
 '공포번호': '00010',
 '언어': '한글',
 '법종구분': '헌법',
 '법령명_한글': '대한민국헌법',
 '법령명_한자': '大韓民國憲法',
 '법령명약칭': None,
 '제명변경여부': 'N',
 '한글법령여부': 'N',
 '편장절관': '01010000',
 '시행일자': '19880225',
 '제개정구분': '전부개정',
 '별표편집여부': 'Y',
 '공포법령여부': 'Y',
 '공동부령정보': None,
 '조문': [{'조문번호': '0',
   '조문여부': '전문',
   '조문제목': None,
   '조문시행일자': '19880225',
   '조문제개정유형': None,
   '조문이동이전': '000000',
   '조문이동이후': '000000',
   '조문변경여부': 'Y',
   '조문내용': ' 전문 '},
  {'조문번호': '0',
   '조문여부': '전문',
   '조문제목': None,
   '조문시행일자': '19880225',
   '조문제개정유형': None,
   '조문이동이전': '000000',
   '조문이동이후': '000000',
   '조문변경여부': 'Y',
   '조문내용': ' 유구한 역사와 전통에 빛나는 우리 대한국민은 3ㆍ1운동으로 건립된 대한민국임시정부의 법통과 불의에 항거한 4ㆍ19민주이념을 계승하고, 조국의 민주개혁과 평화적 통일의 사명에 입각하여 정의ㆍ인도와 동포애로써 민족의 단결을 공고히 하고, 모든 사회적 폐습과 불의를 타파하며, 자율과 조화를 바탕으로 자유민주적 기본질서를 더욱 확고히 하여 정치ㆍ경제ㆍ사회ㆍ문화의 모든 영역에 있어서 각인의 기회를 균등히 하고, 능력을 최고도로 발휘하게 하며, 자유와 권리에 따르는 책임과 의무를 완수하게 하여, 안으로는 국민생활의 균등한 향상을 기하고 밖으로는 항구적인 세계평화와 인류공영에 이바지함으로써 우리들과 우리들의 자손의 안전과 자유와 행복을 영원히 확

In [196]:
get_law_text_by_law_name('../../data/panre/law_text_cleansed', '헌법')

{'법령ID': '001444',
 '공포일자': '19871029',
 '공포번호': '00010',
 '언어': '한글',
 '법종구분': '헌법',
 '법령명_한글': '대한민국헌법',
 '법령명_한자': '大韓民國憲法',
 '법령명약칭': None,
 '제명변경여부': 'N',
 '한글법령여부': 'N',
 '편장절관': '01010000',
 '시행일자': '19880225',
 '제개정구분': '전부개정',
 '별표편집여부': 'Y',
 '공포법령여부': 'Y',
 '공동부령정보': None,
 '조문': [{'조문번호': '0',
   '조문여부': '전문',
   '조문제목': None,
   '조문시행일자': '19880225',
   '조문제개정유형': None,
   '조문이동이전': '000000',
   '조문이동이후': '000000',
   '조문변경여부': 'Y',
   '조문내용': ' 전문 '},
  {'조문번호': '0',
   '조문여부': '전문',
   '조문제목': None,
   '조문시행일자': '19880225',
   '조문제개정유형': None,
   '조문이동이전': '000000',
   '조문이동이후': '000000',
   '조문변경여부': 'Y',
   '조문내용': ' 유구한 역사와 전통에 빛나는 우리 대한국민은 3ㆍ1운동으로 건립된 대한민국임시정부의 법통과 불의에 항거한 4ㆍ19민주이념을 계승하고, 조국의 민주개혁과 평화적 통일의 사명에 입각하여 정의ㆍ인도와 동포애로써 민족의 단결을 공고히 하고, 모든 사회적 폐습과 불의를 타파하며, 자율과 조화를 바탕으로 자유민주적 기본질서를 더욱 확고히 하여 정치ㆍ경제ㆍ사회ㆍ문화의 모든 영역에 있어서 각인의 기회를 균등히 하고, 능력을 최고도로 발휘하게 하며, 자유와 권리에 따르는 책임과 의무를 완수하게 하여, 안으로는 국민생활의 균등한 향상을 기하고 밖으로는 항구적인 세계평화와 인류공영에 이바지함으로써 우리들과 우리들의 자손의 안전과 자유와 행복을 영원히 확

In [201]:
get_jomun_by_law_text(get_law_text_by_law_name('../../data/panre/law_text_cleansed', '헌법'), 54)

' 제54조 '

In [203]:
get_hang_by_law_text(get_law_text_by_law_name('../../data/panre/law_text_cleansed', '헌법'), 54, 3)

' ③새로운 회계연도가 개시될 때까지 예산안이 의결되지 못한 때에는 정부는 국회에서 예산안이 의결될 때까지 다음의 목적을 위한 경비는 전년도 예산에 준하여 집행할 수 있다. '

In [199]:
get_ho_by_law_text(get_law_text_by_law_name('../../data/panre/law_text_cleansed', '헌법'), 54, 3, 1)

' 1. 헌법이나 법률에 의하여 설치된 기관 또는 시설의 유지ㆍ운영 '

In [200]:
get_contents_by_law_text(get_law_text_by_law_name('../../data/panre/law_text_cleansed', '헌법'), 54)

(' 제54조 ',
 [(' ①국회는 국가의 예산안을 심의ㆍ확정한다. ', []),
  (' ②정부는 회계연도마다 예산안을 편성하여 회계연도 개시 90일 전까지 국회에 제출하고, 국회는 회계연도 개시 30일 전까지 이를 의결하여야 한다. ',
   []),
  (' ③새로운 회계연도가 개시될 때까지 예산안이 의결되지 못한 때에는 정부는 국회에서 예산안이 의결될 때까지 다음의 목적을 위한 경비는 전년도 예산에 준하여 집행할 수 있다. ',
   [' 1. 헌법이나 법률에 의하여 설치된 기관 또는 시설의 유지ㆍ운영 ',
    ' 2. 법률상 지출의무의 이행 ',
    ' 3. 이미 예산으로 승인된 사업의 계속 '])])

In [278]:
find_law_section_by_content(law_text_list, " ①국무위원은 국무총리의 제청으로 대통령이 임명한다. ")

('001444', '대한민국헌법', '87', '①', None)

## EDA

In [13]:
import pandas as pd

panre_text = pd.read_csv('../../data/panre/panre_text_cleaned.csv')

In [ ]:
# def remove_empty_from_nested_list(nested_list):
#     # 재귀적으로 리스트를 검사하고, 빈 요소가 아닌 것만 포함하는 새 리스트를 반환
#     result = []
#     for item in nested_list:
#         if isinstance(item, list):
#             # 재귀적 호출로 리스트의 내부 요소들을 정리
#             cleaned_sublist = remove_empty_from_nested_list(item)
#             # cleaned_sublist가 비어있지 않다면 결과에 추가
#             if cleaned_sublist:
#                 result.append(cleaned_sublist)
#         elif item or item == 0:  # item이 리스트가 아니고, 비어있지 않은 경우 (문자열, 숫자 등)
#             # 숫자 0은 False로 평가되지만 유효한 값이므로 명시적으로 검사
#             result.append(item)
#     return result

# # 첫 번째 항목이 숫자인지 확인하는 함수
# def is_number(item):
#     try:
#         float(item[0])  # 첫 번째 항목을 숫자로 변환 시도
#         return True
#     except ValueError:
#         return False



In [34]:
def check_valid_pos(item):
    # print(item)
    ret = len(item) - 1
    for i in item[::-1]:
        if i is None:
            ret -= 1
        else:
            return ret
    return -1
            

# 법령명이 없는 항목을 찾아 이전 법령명을 채워주는 함수
def fill_missing_laws(laws):
    for k in range(len(laws)):
        for i in range(0, len(laws[k]) - 1):
            for j in range(0, check_valid_pos(laws[k][i])):
                if laws[k][i + 1][j] is None:
                    laws[k][i + 1][j] = laws[k][i][j]
    return laws


def remove_unnecessary_texts(law_list):
    for idx1, i in enumerate(law_list):
        for idx2, j in enumerate(i):
            # print("j:", j)
            tmp = check_valid_pos(law_list[idx1][idx2]) + 1
            law_list[idx1][idx2] = law_list[idx1][idx2][:tmp]
    return law_list


def remove_and_report_single_length_sublists(arrays):
    filtered_arrays = []
    removal_count = 0  # 제거된 배열의 수를 세는 변수

    for sublist in arrays:
        before_count = len(sublist)
        # 길이가 1보다 큰 배열만 필터링
        filtered_sublist = [item for item in sublist if isinstance(item, list) and len(item) > 1]
        after_count = len(filtered_sublist)
        removal_count += before_count - after_count  # 제거된 배열의 수 계산
        filtered_arrays.append(filtered_sublist)

    # 결과 메시지 출력
    if removal_count > 0:
        print(f"Warning: {removal_count}개의 배열이 제거되었습니다.")
    
    return filtered_arrays

In [35]:
import re

def clean_text(text):
    """
    텍스트에서 불필요한 문자를 제거하는 함수
    input:
    - text: 정제할 텍스트

    output:
    - 정제된 텍스트 list
    """
    
    # 텍스트에서 숫자와 대괄호를 제외한 모든 문자를 제거하고 /로 나누어 리스트로 변환
    txt = re.sub(r'\[.*?\]', '', re.sub(r'\(.*?\)', '', re.sub('<.*?>', ' / ', text))).split(" / ")
    # 대괄호로 묶인 숫자를 제거
    txt = [re.sub(r'\[\d+\]', '', i) for i in txt]
    txt = [i.replace('\n', '') for i in txt]
    txt = [i.strip() for i in txt]
    for i in range(len(txt)):
        txt[i] = txt[i].split(", ")
        for j in range(len(txt[i])):
            txt[i][j] = txt[i][j].split(" ")
    # print(txt)
    
    # print("처리 전:", txt)
    for i in range(len(txt)):
        for j in range(len(txt[i])): # ... [['민법', '제192조'] ...]
            entry = [None] * 4
            for k in range(len(txt[i][j])-1, -1, -1):
                if txt[i][j][k].startswith("제") and (txt[i][j][k].endswith("조") or txt[i][j][k].endswith("항") or txt[i][j][k].endswith("호")):
                    if txt[i][j][k].endswith("조"): # 제00조
                        entry[1] = re.sub(r'제|조', '', txt[i][j][k])
                    elif txt[i][j][k].endswith("항"):
                        entry[2] = re.sub(r'제|항', '', txt[i][j][k])
                    elif txt[i][j][k].endswith("호"):
                        entry[3] = re.sub(r'제|호', '', txt[i][j][k])
                else: # 아닌 경우 제조항호가 아님 -> 법률이므로 합함
                    for l in range(1, k + 1):
                        txt[i][j][0] += " " + txt[i][j][l]
                    entry[0] = txt[i][j][0]
                    break
            # print("entry:", entry)
            txt[i][j] = entry
    
    txt = fill_missing_laws(txt)
    txt = remove_unnecessary_texts(txt)
    txt = remove_and_report_single_length_sublists(txt)
    
    return txt

In [36]:
for i in range(5, 10):
    print(panre_text.iloc[i]["참조조문"])
    print(clean_text(panre_text.iloc[i]["참조조문"]))
    print()

 [1] 형법 제319조 제1항 / [2] 형법 제319조 제1항 / [3] 형법 제319조 제1항

[[['형법', '319', '1']], [['형법', '319', '1']], [['형법', '319', '1']]]

 [1] 민법 제105조 / [2] 민법 제105조

[[['민법', '105']], [['민법', '105']]]

 [1] 형법 제319조 제1항 / [2] 형법 제319조 제1항

[[['형법', '319', '1']], [['형법', '319', '1']]]

 [1] 소액사건심판법 제3조 제2호 / [2] 민사집행법 제246조 제1항 제8호, 제3항, 구 민사집행법 시행령(2019. 3. 5. 대통령령 제29603호로 개정되기 전의 것) 제7조, 민사소송법 제288조[증명책임]

[[['소액사건심판법', '3', None, '2']], [['민사집행법', '246', '1', '8'], ['민사집행법', '246', '3'], ['구 민사집행법 시행령', '7'], ['민사소송법', '288']]]

교원의 지위 향상 및 교육활동 보호를 위한 특별법 제1조, 제9조 제1항, 제10조, 제10조의2, 행정소송법 제12조

[[['교원의 지위 향상 및 교육활동 보호를 위한 특별법', '1'], ['교원의 지위 향상 및 교육활동 보호를 위한 특별법', '9', '1'], ['교원의 지위 향상 및 교육활동 보호를 위한 특별법', '10'], ['행정소송법', '12']]]



In [37]:
panre_text['참조조문_정제'] = panre_text['참조조문'].dropna().apply(clean_text)

In [39]:
panre_text.to_csv('../../data/panre/panre_text_jomun_cleansed.csv', index=False)

In [53]:
def get_law_text_by_cleaned_jomun_list(cleaned_jomun_list):
    picked_law_text_list = []
    for jomuns in cleaned_jomun_list:
        for jomun in jomuns:
            try:
                if len(jomun) == 2:
                    picked_law_text_list.append(get_contents_by_law_text(get_law_text_by_law_name(LAW_TEXT_DIR, jomun[0]), int(jomun[1]) if jomun[1] is not None else None))
                if len(jomun) >= 3:
                    picked_law_text_list.append(get_contents_by_law_text(get_law_text_by_law_name(LAW_TEXT_DIR, jomun[0]), int(jomun[1]) if jomun[1] is not None else None, int(jomun[2]) if jomun[2] is not None else None))
            except Exception as e:
                pass
                # print(f"처리하지 못했습니다. {e}  미처리 내용: {jomun}")
    return picked_law_text_list

In [54]:
panre_text.iloc[0]["참조조문_정제"]

[[['민법', '192'], ['민법', '195'], ['민법', '758', '1']],
 [['민법', '192'], ['민법', '195'], ['민법', '758', '1']],
 [['실화책임에 관한 법률', '1'], ['실화책임에 관한 법률', '2']],
 [['구 자본시장과 금융투자업에 관한 법률', '64', '1'],
  ['구 자본시장과 금융투자업에 관한 법률', '80', '2'],
  ['신탁법', '114', '1']]]

In [55]:
get_law_text_by_cleaned_jomun_list(panre_text.iloc[0]["참조조문_정제"])

[('제192조(점유권의 취득과 소멸)\n',
  [(' ①물건을 사실상 지배하는 자는 점유권이 있다.\n\t\t\t\t', []),
   ('  ②점유자가 물건에 대한 사실상의 지배를 상실한 때에는 점유권이 소멸한다. 그러나 제204조의 규정에 의하여 점유를 회수한 때에는 그러하지 아니하다.\n\t\t\t\t',
    [])]),
 ('제195조(점유보조자) 가사상, 영업상 기타 유사한 관계에 의하여 타인의 지시를 받어 물건에 대한 사실상의 지배를 하는 때에는 그 타인만을 점유자로 한다.\n',
  []),
 (' ①공작물의 설치 또는 보존의 하자로 인하여 타인에게 손해를 가한 때에는 공작물점유자가 손해를 배상할 책임이 있다. 그러나 점유자가 손해의 방지에 필요한 주의를 해태하지 아니한 때에는 그 소유자가 손해를 배상할 책임이 있다.\n\t\t\t\t',
  []),
 ('제192조(점유권의 취득과 소멸)\n',
  [(' ①물건을 사실상 지배하는 자는 점유권이 있다.\n\t\t\t\t', []),
   ('  ②점유자가 물건에 대한 사실상의 지배를 상실한 때에는 점유권이 소멸한다. 그러나 제204조의 규정에 의하여 점유를 회수한 때에는 그러하지 아니하다.\n\t\t\t\t',
    [])]),
 ('제195조(점유보조자) 가사상, 영업상 기타 유사한 관계에 의하여 타인의 지시를 받어 물건에 대한 사실상의 지배를 하는 때에는 그 타인만을 점유자로 한다.\n',
  []),
 (' ①공작물의 설치 또는 보존의 하자로 인하여 타인에게 손해를 가한 때에는 공작물점유자가 손해를 배상할 책임이 있다. 그러나 점유자가 손해의 방지에 필요한 주의를 해태하지 아니한 때에는 그 소유자가 손해를 배상할 책임이 있다.\n\t\t\t\t',
  []),
 ('제1조(목적) 이 법은 실화(失火)의 특수성을 고려하여 실화자에게 중대한 과실이 없는 경우 그 손해배상액의 경감(輕減)에 관한 「민법」 제765조의 특례를 정함을 목적으로 한다.\n',
  []),
 ('제2조(적용범위)

# Apply Data

In [56]:
from tqdm import tqdm

tqdm.pandas()
panre_text['참조조문_정제_본문'] = panre_text['참조조문_정제'].dropna().progress_apply(get_law_text_by_cleaned_jomun_list)

  0%|          | 0/71818 [00:00<?, ?it/s]

100%|██████████| 71818/71818 [3:08:08<00:00,  6.36it/s]   


In [57]:
panre_text.to_csv('../../data/panre/panre_text_jomun_cleansed_with_law.csv', index=False)

In [72]:
print(panre_text.iloc[18]["참조조문_정제"])

[[['소액사건심판법', '3', '2']], [['약관의 규제에 관한 법률', '5']], [['약관의 규제에 관한 법률', '5'], ['구 국민건강보험법', '44', '2'], ['구 국민건강보험법 시행령', '22', '1'], ['구 국민건강보험법 시행령', '3']], [['민사소송법', '429'], ['민사소송법', '431']]]


In [ ]:
panre_text['참조조문_정제_본문'] = panre_text['참조조문_정제'].dropna().apply(get_law_text_by_cleaned_jomun_list)

In [59]:
panre_text.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87067 entries, 0 to 87066
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   판례정보일련번호    87067 non-null  int64  
 1   사건명         87057 non-null  object 
 2   사건번호        87067 non-null  object 
 3   선고일자        87067 non-null  float64
 4   선고          87066 non-null  object 
 5   법원명         87067 non-null  object 
 6   법원종류코드      87066 non-null  float64
 7   사건종류명       87064 non-null  object 
 8   사건종류코드      87064 non-null  float64
 9   판결유형        87067 non-null  object 
 10  판시사항        72994 non-null  object 
 11  판결요지        63590 non-null  object 
 12  참조조문        71818 non-null  object 
 13  참조판례        40591 non-null  object 
 14  판례내용        87067 non-null  object 
 15  참조조문_정제     71818 non-null  object 
 16  참조조문_정제_본문  71818 non-null  object 
dtypes: float64(3), int64(1), object(13)
memory usage: 11.3+ MB


----